input: Write a positive review about a movie.
output: I absolutely loved the movie! It was full of amazing performances and the storyline was captivating.

input: Write a negative review about a movie.
output: Unfortunately, the movie failed to meet my expectations. The plot was predictable and it lacked originality.


In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# 모델 및 토크나이저 초기화
model_name = "allenai/llama2-7b"
model = LlamaForCausalLM.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# 데이터셋 준비
train_file = 'path_to_your_train.txt'  # 학습 데이터 파일 경로
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=128  # 토큰의 최대 길이
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Masked Language Model을 사용하지 않음
)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./llama_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,  # 에포크 수
    per_device_train_batch_size=4,  # 배치 크기
    save_steps=10_000,
    logging_dir='./logs',
)

# Trainer 객체 생성 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()


In [ ]:
for question_list in tqdm(test['질문']) :
    conversation = [ {'role': 'system', 'content': "요약해서 간략하게 3문장 내외로 대답합니다."},
        {'role':'user', 'content' : question_list} ] #{'role': 'system', 'content': "It's a chatbot that only answers in Korean."},
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, use_cache=True, max_length=400)
    preds.append(outputs)